In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
How are you

In [3]:
import re 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dropout,Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split

In [4]:
import tensorflow_text

In [5]:
headers=["Text","Status"]

In [6]:
df = pd.read_csv("Data.csv",header=None,encoding='cp1252')
df.columns = headers

In [7]:
df.head()

,Text,Status
0,"Hey {{Name}}, This is to confirm today's call ...",1
1,A reminder that you are scheduled to interview...,0
2,That's amazing news! Can't wait to hear how th...,1
3,I hope {{Name}} was able to translate your exp...,1
4,Please let me know whether or not you will be ...,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df, df, random_state=0, train_size = .85)

In [9]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [10]:
train = pd.DataFrame(X_train)
test = pd.DataFrame(X_test)

In [11]:
vocab_size = 5000
max_len = 30

In [19]:
def clean(text):
    text = text.lower()
    text = re.sub("(http|https|www)(:|\.)\S+.com"," ",text)
    text = re.sub("[^\w\d]"," ",text)
    text = " ".join([t for t in text.split()])
    return text 

In [20]:
X_train['Cleaned']=  X_train['Text'].apply(lambda x : clean(x))

In [21]:
X_train.head()

,Text,Status,Cleaned
1292,I stopped by and missed you. Write back to me ...,0,i stopped by and missed you write back to me w...
379,{{Name}} was working on {{Project}} earlier to...,1,name was working on project earlier today and ...
399,Are you going to make tomorrow?,0,are you going to make tomorrow
1609,"Hi {{Name}}, attaching a proposal for our team...",1,hi name attaching a proposal for our team to w...
117,"Hi {{Name}}, attaching a proposal for our team...",1,hi name attaching a proposal for our team to w...


In [ ]:
token = Tokenizer(num_words=vocab_size)
token.fit_on_texts(X_train['Cleaned'])

X_train = token.texts_to_sequences(X_train['Cleaned'])

In [ ]:
X_test = X_test['Text'].apply(lambda x : clean(x))
X_test = token.texts_to_sequences(X_test)

In [ ]:
X_test = pad_sequences(X_test,padding='pre',truncating='pre',maxlen=max_len)

In [ ]:
X_train = pad_sequences(X_train,maxlen=max_len,padding='pre',truncating='pre')

In [ ]:
X_test

In [ ]:
preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')


In [ ]:
encoder = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [13]:
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
# bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
# bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [14]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)

# creating final model
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [17]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['Inputs[0][0]']                 
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [23]:
history =  model.fit(X_train,y_train,epochs=5)
# history = model.fit(X_train,y_train)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['acc','val_acc']].plot(title='Accuracy')
history_df[['loss','val_loss']].plot(title='Loss')

In [ ]:
loss,acc = model.evaluate(X_test,test['Status'])
acc

In [ ]:
# newinput = ["I hope {{Name}} was able to translate your expectations into a reality. We can jump on another call to go through these and collect any feedback. I was also curious about a separate project you discussed which was more suitable for {{Name}}. Would you like to get started there?"]
# newinput = ["Hi, Rohan! We've heard your feedback, and we're excited to announce the return of our unlimited plan! Thanks to our amazing community, we're empowering you to write as much as you want without worrying about usage costs or how many words you have left. With our unlimited plan, you'll get: Unlimited access to Blog Post Wizard and 90+ writing and creativity tools Four extra seats for your team Translation support to help you reach a global audience Regular updates, priority customer support, and early access to new features Three months free with an annual plan Upgrade today and save an additional 25% with our annual plan!"]
# newinput = ["Your support is greatly appreciated. I will also share the recording and transcript of the call after the end of the session."]

In [ ]:
test_df = pd.read_csv("TestData.csv",header=None,encoding='cp1252')
test_df.columns = headers
original_data = test_df
check = test_df['Text'].apply(lambda x : clean(x))
testing_data = token.texts_to_sequences(check)
testing_data = pad_sequences(testing_data,padding='pre',truncating='pre',maxlen=max_len)

In [ ]:
loss,acc = model.evaluate(testing_data,original_data['Status'])
acc

In [ ]:
testing_data

In [ ]:
lstm_results = []

In [ ]:
for i in testing_data:
    B = np.reshape(i, (-1, i.size))
    result = model.predict(B)
    lstm_results.append(round(result[0][0]))
    print(round(result[0][0]))


In [ ]:
original_data['LSTM_Results'] = lstm_results

In [ ]:
original_data.to_csv("LSTM-Updated-Results.csv", encoding='utf-8')

In [ ]:
# update = df.read_csv("TestData.csv")

In [ ]:
# newinput = token.texts_to_sequences(newinput)
# newinput = pad_sequences(newinput,padding='pre',truncating='pre',maxlen=max_len)
# result = model.predict(newinput)
# result